In [1]:
from ants import *

In [4]:
a = 0
if not a:
    print('f')

f


# Problem 4
## Unlock Questions
### Q1:
Q: What class do ShortThrower and LongThrower inherit from?
Choose the number of the correct choice:
* 0) Bee
* 1) ShortThrower
* 2) ThrowerAnt
* 3) LongThrower

A: 2)

### Q2
Q: What constraint does a regular ThrowerAnt have on its throwing distance?
Choose the number of the correct choice:
* 0) There is no restriction on how far a regular ThrowerAnt can throw
* 1) A regular ThrowerAnt can only attack Bees at most 3 places away
* 2) A regular ThrowerAnt can only attack Bees at most 5 places away
* 3) A regular ThrowerAnt can only attack Bees at least 3 places away

A: 0)

### Q3
Q: What constraint does a LongThrower have on its throwing distance?
Choose the number of the correct choice:
* 0) A LongThrower can only attack Bees at most 5 places away
* 1) There is no restriction on how far a LongThrower can throw
* 2) A LongThrower can only attack Bees at least 5 places away
* 3) A LongThrower can only attack Bees at least 3 places away

A: 2)

### Q4
Q: What constraint does a ShortThrower have on its throwing distance?
Choose the number of the correct choice:
* 0) A ShortThrower can only attack Bees at most 5 places away
* 1) A ShortThrower can only attack Bees at least 3 places away
* 2) There is no restriction on how far a ShortThrower can throw
* 3) A ShortThrower can only attack Bees at most 3 places away

A: 3)

### Q5
Q: With the addition of these new ThrowerAnt subclasses, we must modify
our definition of nearest_bee. Now what Bee should ThrowerAnts throw at?
Choose the number of the correct choice:
* 0) The closest Bee behind it within range
* 1) Any Bee within range
* 2) The closest Bee in front of it within range
* 3) Any Bee in its current Place

A: 2)

### Q6
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing Long/ShortThrower parameters
>>> ShortThrower.food_cos #?
```
A: 2

### Q7
```python
>>> LongThrower.food_cost
```
A: 2

### Q8
```python
>>> short_t = ShortThrower()
>>> long_t = LongThrower()
>>> short_t.health
```
A: 1

### Q9
```python
>>> long_t.health
```
A: 1

# Important base classes

In [2]:
# Place class
class Place:
    """A Place holds insects and has an exit to another Place."""
    is_hive = False

    def __init__(self, name, exit=None):
        """Create a Place with the given NAME and EXIT.

        name -- A string; the name of this Place.
        exit -- The Place reached by exiting this Place (may be None).
        """
        self.name = name
        self.exit = exit
        self.bees = []        # A list of Bees
        self.ant = None       # An Ant
        self.entrance = None  # A Place
        # Phase 1: Add an entrance to the exit
        # BEGIN Problem 2
        "*** YOUR CODE HERE ***"
        if self.exit:
            self.exit.entrance = self
        # END Problem 2

    def add_insect(self, insect):
        """
        Asks the insect to add itself to the current place. This method exists so
            it can be enhanced in subclasses.
        """
        insect.add_to(self)

    def remove_insect(self, insect):
        """
        Asks the insect to remove itself from the current place. This method exists so
            it can be enhanced in subclasses.
        """
        insect.remove_from(self)

    def __str__(self):
        return self.name
    
# Insect class    
class Insect:
    """An Insect, the base class of Ant and Bee, has health and a Place."""

    damage = 0
    # ADD CLASS ATTRIBUTES HERE

    def __init__(self, health, place=None):
        """Create an Insect with a health amount and a starting PLACE."""
        self.health = health
        self.place = place  # set by Place.add_insect and Place.remove_insect

    def reduce_health(self, amount):
        """Reduce health by AMOUNT, and remove the insect from its place if it
        has no health remaining.

        >>> test_insect = Insect(5)
        >>> test_insect.reduce_health(2)
        >>> test_insect.health
        3
        """
        self.health -= amount
        if self.health <= 0:
            self.death_callback()
            self.place.remove_insect(self)

    def action(self, gamestate):
        """The action performed each turn.

        gamestate -- The GameState, used to access game state information.
        """

    def death_callback(self):
        # overriden by the gui
        pass

    def add_to(self, place):
        """Add this Insect to the given Place

        By default just sets the place attribute, but this should be overriden in the subclasses
            to manipulate the relevant attributes of Place
        """
        self.place = place

    def remove_from(self, place):
        self.place = None

    def __repr__(self):
        cname = type(self).__name__
        return '{0}({1}, {2})'.format(cname, self.health, self.place)
    
class Bee(Insect):
    """A Bee moves from place to place, following exits and stinging ants."""

    name = 'Bee'
    damage = 1
    # OVERRIDE CLASS ATTRIBUTES HERE

    def sting(self, ant):
        """Attack an ANT, reducing its health by 1."""
        ant.reduce_health(self.damage)

    def move_to(self, place):
        """Move from the Bee's current Place to a new PLACE."""
        self.place.remove_insect(self)
        place.add_insect(self)

    def blocked(self):
        """Return True if this Bee cannot advance to the next Place."""
        # Special handling for NinjaAnt
        # BEGIN Problem Optional 1
        return self.place.ant is not None
        # END Problem Optional 1

    def action(self, gamestate):
        """A Bee's action stings the Ant that blocks its exit if it is blocked,
        or moves to the exit of its current place otherwise.

        gamestate -- The GameState, used to access game state information.
        """
        destination = self.place.exit

        # Extra credit: Special handling for bee direction
        if self.blocked():
            self.sting(self.place.ant)
        elif self.health > 0 and destination is not None:
            self.move_to(destination)

    def add_to(self, place):
        place.bees.append(self)
        Insect.add_to(self, place)

    def remove_from(self, place):
        place.bees.remove(self) # Remove from bee list
        Insect.remove_from(self, place)

## Coding Solutions
### Solution_partA: Updated Base ThrowerAnt Class

In [3]:
class ThrowerAnt(Ant):
    """ThrowerAnt throws a leaf each turn at the nearest Bee in its range."""

    name = 'Thrower'
    implemented = True
    damage = 1
    food_cost = 3
    
    max_range = float("inf") # New class variable ranges, 
    min_range = 0 # we track curr_location with "range" attributes and "step" in find_nearest()

    def nearest_bee(self):
        """Return the nearest Bee in a Place that is not the HIVE, connected to
        the ThrowerAnt's Place by following entrances.

        This method returns None if there is no such Bee (or none in range).
        """
        def find_nearest(myplace, step): # 
            if myplace.is_hive: # Basecase: curr_place is hive, then returns None
                return None
            if self.min_range <= step <= self.max_range: # Within the range, check bees and return
                if myplace.bees:
                    return random_bee(myplace.bees)
                else: # If no bee, return recursive function
                    return find_nearest(myplace.entrance, step + 1)

            if step < self.min_range: # If 'place' is still below the minimum range, return recursive calls on next place
                return find_nearest(myplace.entrance, step + 1)
            else: # If 'place' goes beyond range, return false 
                return None
        curr_place = self.place
        
        return find_nearest(curr_place, 0)

**Note:** Return recursive function, or call recursive function

**Example**: A "correct" recursive prime number judger

In [4]:
def isprime(n):
    def helper(n, div):
        if div > n ** 0.5:
            return True
        else:
            if n % div == 0:
                return False
            else:
                return helper(n, div + 1)
    return helper(n, 2)

In [5]:
isprime(97)

True

If we make the recursive function call but not returning its result, we will get None

In [6]:
def isprime(n):
    def helper(n, div):
        if div > n ** 0.5:
            return True
        else:
            if n % div == 0:
                return False
            else:
                helper(n, div + 1)
    return helper(n, 2)

In [7]:
print(isprime(97))

None


### Solution_partB: ShortThrower and LongThrower classes
As we got a nicely implemented base method, what we need to do is just setting up the overriden class attributes correctly 

In [8]:
class ShortThrower(ThrowerAnt):
    """A ThrowerAnt that only throws leaves at Bees at most 3 places away."""

    name = 'Short'
    food_cost = 2
    # OVERRIDE CLASS ATTRIBUTES HERE
    max_range = 3
    # BEGIN Problem 4
    implemented = True   # Change to True to view in the GUI
    # END Problem 4


class LongThrower(ThrowerAnt):
    """A ThrowerAnt that only throws leaves at Bees at least 5 places away."""

    name = 'Long'
    food_cost = 2
    # OVERRIDE CLASS ATTRIBUTES HERE
    min_range = 5
    # BEGIN Problem 4
    implemented = True   # Change to True to view in the GUI
    # END Problem 4

# Problem 5
## Unlock Questions
## Overview
### Q1
Q: How can you obtain the current place of a FireAnt?
Choose the number of the correct choice:
* 0) By calling the FireAnt constructor
* 1) By accessing the place instance attribute, which is the name of
   some Place object
* 2) By accessing the place instance attribute, which is a Place object
* 3) By calling the Place constructor, passing in the FireAnt instance

A: 2)

### Q2
Q: How can you obtain all of the Bees currently in a given place?
Choose the number of the correct choice:
* 0) By calling the Bee constructor, passing in the place instance
* 1) By accessing the bees instance attribute, which is a dictionary of
   Bee objects
* 2) By calling the add_insect method on the place instance
* 3) By accessing the bees instance attribute, which is a list of Bee
   objects
   
A: 3)

### Q3
Q: Can you iterate over a list while mutating it?
Choose the number of the correct choice:
* 0) Yes, you can mutate a list while iterating over it with no problems
* 1) Yes, but you should iterate over a copy of the list to avoid skipping
   elements
* 2) No, Python doesn't allow list mutation on a list that is being
   iterated through
   
A: 1)

## Fire class attributes
### Q4
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing FireAnt parameters
>>> fire = FireAnt()
>>> FireAnt.food_cost
```
A: 5

### Q5
```python
>>> fire.health
```
A: 3



## Interact between fire and bee
### Q6
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing fire does damage to all Bees in its Place
>>> place = gamestate.places['tunnel_0_4']
>>> fire = FireAnt(health=1)
>>> place.add_insect(fire)        # Add a FireAnt with 1 health
>>> place.add_insect(Bee(3))      # Add a Bee with 3 health
>>> place.add_insect(Bee(5))      # Add a Bee with 5 health
>>> len(place.bees)               # How many bees are there?
```

A: 2

### Q7
```python
>>> place.bees[0].action(gamestate)  # The first Bee attacks FireAnt
>>> fire.health ## Note this fire has a health of 1 
```
A: 1

### Q8
```python
>>> fire.place is None # Fire died, and place.fire == None 
```
A: True

### Q9
```python
>>> len(place.bees)
```

A: 1 

### Q10
```python
>>> place.bees[0].health 
```
A: 1

## Methods ```Place.remove_insect()``` and ```Insect,remove_From()```
### Q11
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> place = gamestate.places['tunnel_0_4']
>>> ant = FireAnt(health=1)           # Create a FireAnt with 1 health
>>> place.add_insect(ant)      # Add a FireAnt to place
>>> ant.place is place
```
A: True

### Q12
```python
>>> place.remove_insect(ant)   # Remove FireAnt from place
>>> ant.place is place         # Is the ant's place still that place?
```
A: True

```python
[Bee(3, tunnel_0_0)]
[Bee(3, tunnel_0_0)]
2
Bee(3, tunnel_0_0) 3
[Bee(2, tunnel_0_0)]
[Bee(2, tunnel_0_0)]
1
Bee(2, tunnel_0_0) 2
[Bee(1, tunnel_0_0)]
[Bee(1, tunnel_0_0)]
0
Bee(1, tunnel_0_0) 1
insect died
insect died
```

# Coding Solutions

In [9]:
class FireAnt(Ant):
    """FireAnt cooks any Bee in its Place when it expires."""

    name = 'Fire'
    damage = 3
    food_cost = 5
    # OVERRIDE CLASS ATTRIBUTES HERE
    # BEGIN Problem 5
    implemented = True   # Change to True to view in the GUI
    # END Problem 5

    def __init__(self, health=3):
        """Create an Ant with a HEALTH quantity."""
        super().__init__(health)

    def reduce_health(self, amount):
        """Reduce health by AMOUNT, and remove the FireAnt from its place if it
        has no health remaining.

        Make sure to reduce the health of each bee in the current place, and apply
        the additional damage if the fire ant dies.
        """
        # BEGIN Problem 5
        "*** YOUR CODE HERE ***"
        # Style 0: Use an intermediate variable
        self.health = self.health - amount
        bees_cpy = self.place.bees[:]
        for bee in bees_cpy:
            bee.reduce_health(amount)
        bees_cpy = self.place.bees[:] # Note basecpy should be updated after reduce_health call
        if self.health <= 0:
            for bee in bees_cpy:
                # print(bee)
                bee.reduce_health(self.damage)

        # Style 1: Inplace style
        for bee in self.place.bees[:]:
            print(bee, bee.health)
            bee.reduce_health(amount)
        if self.health <= 0:
            for bee in self.place.bees[:]:
                print(bee)
                bee.reduce_health(self.damage)
        
        super().reduce_health(0)
        # END Problem 5

In [13]:
l = [1, 2 ,3]
for e in l:
    e = e * 2
print(l)

for i in range(len(l)):
    l[i] *= 2
print(l)

[1, 2, 3]
[2, 4, 6]
